# Исследование надежности заемщиков

**Цель исследования:** На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок


# Обзор данных

In [1]:
import pandas as pd# импортируем библиотеку pandas

In [2]:
# Считаем данные из csv-файла в датафрейм и сохраним в переменную data.
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')


In [3]:
print(data.head(20)) # Выведем первые 20 строчек датафрейма data на экран.

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

In [4]:
#Выведим основную информацию о датафрейме с помощью метода info() 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


# Предобработка данных

## Удаление пропусков

In [5]:
# Выведим количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


 В двух столбцах есть пропущенные значения. Один из них — days_employed. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type. 

In [6]:
for type in data['income_type'].unique():
    data.loc[(data['income_type'] == type) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == type),  'total_income'].median()


## Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода abs().

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
# Для каждого типа занятости выведиv медианное значение трудового стажа days_employed в днях.
print(data.groupby('income_type')['days_employed'].median())

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

In [9]:
#ыведим перечень уникальных значений столбца children.
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


В столбце children есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма data

In [10]:
data = data[(data.children != 20) & (data.children >= 0)]

In [11]:
#Ещё раз выведим перечень уникальных значений столбца children, чтобы убедиться, что артефакты удалены.
print(data['children'].unique())

[1 0 3 2 4 5]


In [12]:
## Удаление пропусков (продолжение)

Заполним пропуски в столбце days_employed медианными значениями по каждому типу занятости income_type

In [13]:
for type in data['income_type'].unique():
    data.loc[(data['income_type'] == type) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == type),  'days_employed'].median()

In [14]:
#Убедимся, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.
print(data.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


## Изменение типов данных

In [15]:
#Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype()
data['total_income'] = data['total_income'].astype('int')

## Обработка дубликатов

In [16]:
data['education'] = data['education'].str.lower() # приведем к нижнему регистру значения столбца 'education'

In [17]:
print(data.duplicated().sum())# посчитаем дубликаты

71


In [18]:
data = data.drop_duplicates().reset_index(drop=True) # удалим дубликаты

## Категоризация данных

 На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.



In [19]:
def categorize_income(total_income):
    if total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    if total_income > 1000000:
        return 'A'
    
# создаем функцию categorize_income()

In [20]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

# применим функцию методом apply()

In [21]:
print(data['purpose'].unique())#  Выведим на экран перечень уникальных целей взятия кредита из столбца purpose

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


 Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.

In [22]:
def categorize_purpose(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'жиль'in purpose:
        return 'операции с недвижимостью'
    if  'недвижимост' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'

In [23]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

# Исследование данных

## Есть ли зависимость между количеством детей и возвратом кредита в срок?¶

Создаем функцию, которая на основании данных из столбца debt сформирует новый столбец debt_category и разделит данные на две категории: кредитов выплачено в срок и просрочено кредитов. Также разделим заемщиков на категории по количеству детей в семье, и добавим новый столбец children_category.

In [24]:
def categorize_debt(debt):
    try:
        if debt == 0:
            return 'Кредитов_выплачено_в_срок'
        else:
            return 'Просрочено_кредитов'
    except:
        pass
data['debt_category'] = data['debt'].apply(categorize_debt)

def categorize_children(children):
    try:
        if children == 0:
            return 'без_детей'
        elif children == 1:
            return '1_ребенок'
        elif children == 2:
            return '2_ребенка'
        elif children > 2:
            return 'многодетные'
    except:
        pass
data['children_category'] = data['children'].apply(categorize_children)

In [25]:
data_children = data.pivot_table(index='children_category', columns='debt_category', values='debt', aggfunc='count')
#создаем сводную таблицу data_children
data_children['Выдано_кредитов'] = data_children[['Кредитов_выплачено_в_срок', 'Просрочено_кредитов']].sum(axis=1)
#добавляем столбец Выдано_кредитов с суммой всех кредитов просроченных и нет по категориям

In [26]:

#методом .assign() создаем новый столбец в таблице data_children, в котором указываем долю в % просроченных кредитов от общего числа выданных кредитов
data_children = data_children.assign(Доля_просроченных_кредитов = (data_children['Просрочено_кредитов']/(data_children['Просрочено_кредитов'] + data_children['Кредитов_выплачено_в_срок']))*100)
data_children = data_children.sort_values(by='Доля_просроченных_кредитов', ascending=True)
display(data_children
        .style
        .format('{:.2f}%', subset='Доля_просроченных_кредитов')
        .background_gradient(subset=['Доля_просроченных_кредитов'], cmap='Reds')
       )

debt_category,Кредитов_выплачено_в_срок,Просрочено_кредитов,Выдано_кредитов,Доля_просроченных_кредитов
children_category,,,,
без_детей,13028,1063,14091,7.54%
многодетные,349,31,380,8.16%
1_ребенок,4364,444,4808,9.23%
2_ребенка,1858,194,2052,9.45%


In [27]:
# считаем количество всех выданных кредитовЮ попавших в выборку
total = data_children['Выдано_кредитов'].sum()
display(total)

21331

In [28]:
display('Доля кредитов, выданных заемщикам с тремя и более детьми, составляет {:.2%}'.format(380/total))

'Доля кредитов, выданных заемщикам с тремя и более детьми, составляет 1.78%'

**Вывод:**

На основании полученных данных, можно сделать вывод о том, что самый низкий процент невыплаченных в срок кредитов приходится на заемщиков без детей. Далее следуют заемщики из категории "многодетные", имеющие более 3-х детей. Доля кредитов, выданных заемщикам из категории "Многодетные" составляет 1.78 %. Если пренебречь этими данными, то можно сделать вывод о том, что с увеличением количества детей, риск невыплаты кредита в срок увеличивается.

## Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [29]:
data_family_status = data.pivot_table(index='family_status', columns='debt_category', values='debt', aggfunc='count')
data_family_status['Выдано_кредитов'] = data_family_status[['Кредитов_выплачено_в_срок', 'Просрочено_кредитов']].sum(axis=1)
data_family_status = data_family_status.assign(Доля_просроченных_кредитов = (data_family_status['Просрочено_кредитов']/(data_family_status['Просрочено_кредитов'] + data_family_status['Кредитов_выплачено_в_срок']))*100)
data_family_status = data_family_status.sort_values(by='Доля_просроченных_кредитов', ascending=True)
display(data_family_status
        .style
        .format('{:.2f}%', subset='Доля_просроченных_кредитов')
        .background_gradient(subset=['Доля_просроченных_кредитов'], cmap='Reds')
       )


debt_category,Кредитов_выплачено_в_срок,Просрочено_кредитов,Выдано_кредитов,Доля_просроченных_кредитов
family_status,,,,
вдовец / вдова,888,63,951,6.62%
в разводе,1105,84,1189,7.06%
женат / замужем,11334,927,12261,7.56%
гражданский брак,3749,385,4134,9.31%
Не женат / не замужем,2523,273,2796,9.76%


**Вывод:**

Прямой зависимости между семейным статусом заемщика и риском невыплаты кредита в срок нет. Самая высокая доля среди просроченных кредитов находится в категории заемщиков "Не женат/не замужем". Почи каждый 10 кредит из этой категории не будет выплачен в срок.

## Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [30]:
data_total_income = data.pivot_table(index='total_income_category', columns='debt_category', values='debt', aggfunc='count')
data_total_income['Выдано_кредитов'] = data_total_income[['Кредитов_выплачено_в_срок', 'Просрочено_кредитов']].sum(axis=1)
data_total_income = data_total_income.assign(Доля_просроченных_кредитов = (data_total_income['Просрочено_кредитов']/(data_total_income['Просрочено_кредитов'] + data_total_income['Кредитов_выплачено_в_срок']))*100)
data_total_income = data_total_income.sort_values(by='total_income_category', ascending=False)
display(data_total_income
        .style
        .format('{:.2f}%', subset='Доля_просроченных_кредитов')
        .background_gradient(subset=['Доля_просроченных_кредитов'], cmap='Reds')
       )


debt_category,Кредитов_выплачено_в_срок,Просрочено_кредитов,Выдано_кредитов,Доля_просроченных_кредитов
total_income_category,,,,
E,20,2,22,9.09%
D,328,21,349,6.02%
C,14568,1353,15921,8.50%
B,4660,354,5014,7.06%
A,23,2,25,8.00%


**Вывод:**

Согласно проведенному анализу, прямой зависимости между уровнем дохода и риском возникновения просроченной задолженности нет. С увеличением дохода заемщика данный риск не снижается. При этом можно сделать вывод, что самая высокая просрочка наблюдается у заемщиков с доходом менее 30 000,00 рублей в месяц.

## Как разные цели кредита влияют на его возврат в срок?

In [31]:
data_purpose = data.pivot_table(index='purpose_category', columns='debt_category', values='debt', aggfunc='count')
data_purpose['Выдано_кредитов'] = data_purpose[['Кредитов_выплачено_в_срок', 'Просрочено_кредитов']].sum(axis=1)
data_purpose = data_purpose.assign(Доля_просроченных_кредитов = (data_purpose['Просрочено_кредитов']/(data_purpose['Просрочено_кредитов'] + data_purpose['Кредитов_выплачено_в_срок']))*100)
data_purpose = data_purpose.sort_values(by='Доля_просроченных_кредитов', ascending=True)
display(data_purpose
        .style
        .format('{:.2f}%', subset='Доля_просроченных_кредитов')
        .background_gradient(subset=['Доля_просроченных_кредитов'], cmap='Reds')
       )

debt_category,Кредитов_выплачено_в_срок,Просрочено_кредитов,Выдано_кредитов,Доля_просроченных_кредитов
purpose_category,,,,
операции с недвижимостью,9971,780,10751,7.26%
проведение свадьбы,2130,183,2313,7.91%
получение образования,3619,369,3988,9.25%
операции с автомобилем,3879,400,4279,9.35%


**Вывод:**

Согласно данным, указанным в выше размещенной таблице, можно сделать вывод, что самый минимальный риск просрочки у кредитов, выданных на цели "операции с недвижимостью". Чаще всего не возвращают в срок кредиты выданные на цели "получение образования" и "операции с автомобилем".

# Общий вывод

По результатам исследования выявлена следующая закономерность: с ростом количества детей у заемщика, увеличивается риск не выплаты кредита в срок.

Прямой зависмомости между семейным статусом заемщика и риском невыплаты кредита в срок нет. 

При этом самая высокая доля среди просроченных кредитов находится в категории заемщиков "Не женат/не замужем". Почти каждый 10 кредит из этой категории не будет выплачен в срок. 

Прямой зависимости между уровнем дохода и риском возникновения просроченной задолженности нет. С увеличением дохода заемщика данный риск не снижается. 

Самая высокая просрочка наблюдается у заемщиков с доходом менее 30 000,00 рублей в месяц. 

Самый минимальный риск просрочки у кредитов, выданных на цели "операции с недвижимостью". Чаще всего не возвращают в срок кредиты выданные на цели "получение образования" и "операции с автомобилем".

**Рекомендации:** С точки зрения опыта работы в отделе кредитования банка, я бы рекомендовала несколько изменить исследование. Разбить все выданные кредиты из выборки на категории: под залог недвижимости, под залог автомобиля, на потребильские нужды (с подкатегориями: на образование, на свадьбу, на покупку подержанного автомобиля, на ремонт жилья, на строительство недвижимости). Выявить, есть ли закономерность по выше указанным гипотезам согласно вновь выделенным категориям. Углубить исследование, путем выгрузки дополнительных данных о размере долга и ежемесячном платеже. Расчитать коэффициенты платеж/доход, кредит/залог. Проверить есть ли зависимость между значениями коэффициентов п/д, к/д и возвратом кредита в срок, в разрезе каждой категории и в целом по кредитному портфелю.